In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import weather key
from config import wkey
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# url = "http://api.openweathermap.org/data/2.5/weather?"
import requests
import json
# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# # Use the tuple() function to display the latitude and longitude combinations.
# for coordinate in coordinates:
#     print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
#           citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

746

In [6]:
# Starting URL for Weather Map API Call.
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + wkey
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=974c53a57757da54264303801dc0283d


In [7]:
city_url = url + "&q=" + "Boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=974c53a57757da54264303801dc0283d&q=Boston


In [8]:
# Make a 'Get' request for the city weather.
city_weather = requests.get(city_url)
city_weather.status_code

200

In [25]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
#     print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_datw,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind
                          })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping..." + city)
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
City not found. Skipping...saint-philippe
City not found. Skipping...arraial do cabo
City not found. Skipping...nanfeng
City not found. Skipping...marcona
City not found. Skipping...mar del plata
City not found. Skipping...castro
City not found. Skipping...ribeira grande
City not found. Skipping...rikitea
City not found. Skipping...coos bay
City not found. Skipping...lukulu
City not found. Skipping...luorong
City not found. Skipping...roald
City not found. Skipping...pokrovsk-uralskiy
City not found. Skipping...raga
City not found. Skipping...barcelos
City not found. Skipping...sitka
City not found. Skipping...port lincoln
City not found. Skipping...ushuaia
City not found. Skipping...naryan-mar
City not found. Skipping...carnarvon
City not found. Skipping...punta arenas
City not found. Skipping...usinsk
City not found. Skipping...nizhneyansk
City not found. Skipping...nikolskoye
City not found. Skipping...pousat
City not found

City not found. Skipping...robertson
City not found. Skipping...kathu
City not found. Skipping...shimoda
City not found. Skipping...cusuna
City not found. Skipping...yerbogachen
City not found. Skipping...codrington
City not found. Skipping...gazojak
City not found. Skipping...palauig
City not found. Skipping...gbarnga
City not found. Skipping...zharkent
City not found. Skipping...nelson bay
City not found. Skipping...takoradi
City not found. Skipping...yinchuan
City not found. Skipping...kodiak
City not found. Skipping...moerai
City not found. Skipping...puerto leguizamo
City not found. Skipping...mahibadhoo
City not found. Skipping...lake cowichan
City not found. Skipping...iberia
City not found. Skipping...tranas
City not found. Skipping...grand baie
City not found. Skipping...kazaki
City not found. Skipping...ahipara
City not found. Skipping...soe
City not found. Skipping...ipoti
City not found. Skipping...beringovskiy
City not found. Skipping...mahebourg
City not found. Skipping..

City not found. Skipping...teya
City not found. Skipping...safwah
City not found. Skipping...garden city
City not found. Skipping...goksun
City not found. Skipping...qinnan
City not found. Skipping...chirongui
City not found. Skipping...vestmanna
City not found. Skipping...along
City not found. Skipping...barra patuca
City not found. Skipping...north platte
City not found. Skipping...arlit
City not found. Skipping...ashdod
City not found. Skipping...itarema
City not found. Skipping...sola
City not found. Skipping...boma
City not found. Skipping...washougal
City not found. Skipping...ulaangom
City not found. Skipping...bathsheba
City not found. Skipping...panzhihua
City not found. Skipping...gasa
City not found. Skipping...saint anthony
City not found. Skipping...yar-sale
City not found. Skipping...payakumbuh
City not found. Skipping...shingu
City not found. Skipping...miri
City not found. Skipping...fukue
City not found. Skipping...whitehorse
City not found. Skipping...kaputa
City not 

City not found. Skipping...koygorodok
City not found. Skipping...kismayo
City not found. Skipping...kirkuk
City not found. Skipping...samarai
City not found. Skipping...kannauj
City not found. Skipping...wahran
City not found. Skipping...cilegon
City not found. Skipping...atzompa
City not found. Skipping...north bend
City not found. Skipping...acajutla
City not found. Skipping...simpang
City not found. Skipping...sabzevar
City not found. Skipping...warqla
City not found. Skipping...mae ramat
City not found. Skipping...kloulklubed
City not found. Skipping...tanout
City not found. Skipping...bilecik
City not found. Skipping...nhulunbuy
City not found. Skipping...tezpur
City not found. Skipping...ati
City not found. Skipping...charagua
City not found. Skipping...ozgon
City not found. Skipping...dingli
City not found. Skipping...tecolutla
City not found. Skipping...kainantu
City not found. Skipping...hobyo
City not found. Skipping...kitami
City not found. Skipping...urucui
City not found. 

In [28]:
len(city_data)


0

In [27]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

""
